In [1]:
import pandas as pd

In [6]:
import string
import re
import numpy as np
import pandas as pd
from tensorflow import keras
%matplotlib inline
pd.set_option('display.max_colwidth', 200)


In [8]:
import_df = pd.read_csv("ipatest.csv")

In [9]:
array = np.array(import_df)

In [10]:
array

array([['hice', "'i-se"],
       ['combinación', "kom-bi-na-'si̯on"],
       ['inicio', "i-'ni-si̯o"],
       ['educación', "e-ðu-ka-'si̯on"],
       ['hasta', "'as-ta"],
       ['ojo', "'o-ho"],
       ['casi', "'ka-si"],
       ['usar', "u-'saɾ"],
       ['funcionar', "fun-si̯o-'naɾ"],
       ['cantar', "kan-'taɾ"],
       ['ayuda', "a-'ju-ða"],
       ['hombre', "'om-bɾe"],
       ['está', "es-'ta"],
       ['sabe', "'sa-βe"],
       ['oportunidad', "o-poɾ-tu-ni-'ðað"],
       ['toque', "'to-ke"],
       ['él', "'el"],
       ['su', "'su"],
       ['hermana', "eɾ-'ma-na"],
       ['esta', "'es-ta"],
       ['taza', "'ta-sa"],
       ['importante', "im-poɾ-'tan-te"],
       ['qué', "'ke"],
       ['el', "'el"],
       ['tanto', "'tan-to"],
       ['son', "'son"],
       ['masa', "'ma-sa"],
       ['diez', "'di̯es"],
       ['eso', "'e-so"],
       ['éxito', "'ek-si-to"],
       ['extra', "'eks-tɾa"],
       ['miedo', "'mi̯e-ðo"],
       ['voy', "'βoi̯"],
       ['leí', "le-'i"],
    

## In this dataset, the x axis is the Spanish word, the y axis is the ipa transcription with the accents and syllable breaks

## Let's get a better idea of what the string data actually looks like:

In [38]:
# first, let's grab one of the Spanish words
spanish_word = array[45][0]
print("Spanish Word: ", spanish_word)
for i,e in enumerate(spanish_word):
    print(i, e)

Spanish Word:  diario
0 d
1 i
2 a
3 r
4 i
5 o


In [40]:
# let's grab one of the IPA words
ipa_word = array[45][1]
print("IPA Word: ", ipa_word)
for i,e in enumerate(ipa_word):
    print(i,e)

IPA Word:  'di̯a-ɾi̯o
0 '
1 d
2 i
3 ̯
4 a
5 -
6 ɾ
7 i
8 ̯
9 o


## As we can see, the lengths are going to be different. Obviously, there is the addition of the syllable breaks (`-`) and the accent marks (`'`), but there is also the factor of the semi-vowel markings, which are being interpreted as new characters (` ̯`)

In [44]:
# lets test one more, just to see what it might look like
spanish_word2 = array[55][0]
print("Spanish Word: ", spanish_word2)
for i,e in enumerate(spanish_word2):
    print(i, e)

Spanish Word:  patrón
0 p
1 a
2 t
3 r
4 ó
5 n


In [45]:
ipa_word2 = array[55][1]
print("IPA Word: ", ipa_word2)
for i,e in enumerate(ipa_word2):
    print(i,e)

IPA Word:  pa-'tɾon
0 p
1 a
2 -
3 '
4 t
5 ɾ
6 o
7 n


# So it looks like the accented "o" (`ó`) is still read as a single character. This all is good information to know before moving forward